# **GNU Colab**

*In here we set an environment in which you can persistently run desktop programs.*

This notebook lets you **run desktop programs on colab machines** drawn on your local terminal through VNC.

At this moment, the notebook is preconfigured to run either **MATE** or **XFCE**. 

Other options can of course be added.

**Best run**: TBD


## User management

*In here an user `user` is set up.*

Set up a familiar work environment.

**Best run**: 12 seconds

**Worst run**: 35 seconds

In [ ]:
#@title New **user** (default: `user`) { vertical-output: true }
#@markdown Concise. It also enables cron` service.
user = "user" #@param {type:"string"}
password = "testone" #@param {type:"string"}
run = True #@param {type:"boolean"}
root = True #@param {type:"boolean"}


from os import environ

def create_user(user=user, password=password, root=root):
  environ['user'] = user
  environ['password'] = password
  !useradd $user > /dev/null 2>&1
  !echo -e "$password\n$password" | passwd root
  !mkdir /home/$user > /dev/null 2>&1
  !chown -R $user:$user /home/$user
  !touch /var/log/pip.log
  !chown user:user /var/log/pip.log

  if root:
    !echo -e "$password\n$password" | passwd root
  #!python3 -m pip uninstall -y google-colab

if run:
  create_user()

  !apt install cron > /dev/null 2>&1
  !service cron restart

In [ ]:
#@title Install **Oh my zsh!**
#@markdown You don't want to get lost.
user = "user" #@param {type:"string"}
root = True #@param {type:"boolean"}

from os import environ

environ['user'] = user

!apt --quiet install zsh nyancat nyancat-server > /dev/null 2>&1
 
!runuser -l $user -c "$(curl -fsSL https://raw.githubusercontent.com/ohmyzsh/ohmyzsh/master/tools/install.sh) --unattended" > /dev/null 2>&1

!usermod --shell /usr/bin/zsh $user
if root:
  !sh -c "$(curl -fsSL https://raw.githubusercontent.com/ohmyzsh/ohmyzsh/master/tools/install.sh) --unattended" > /dev/null 2>&1
  !usermod --shell /usr/bin/zsh root

In [ ]:
#@title Install a reasonable **text editor** { vertical-output: true }
#@markdown Kidding.
editor = "vim" #@param {type:"string"}
from os import environ

environ['editor'] = editor
!apt --quiet install -y $editor > /dev/null 2>&1

In [ ]:
#@title Install a **window manager**
#@markdown I mean `screen`.

!apt install screen > /dev/null 2>&1

In [ ]:
#@title Install Monitoring { vertical-output: true, form-width: "50px" }
#@markdown I/O and cron logging.

# I/O
!apt install sysstat > /dev/null 2>&1

# Install rsyslog
!apt install rsyslog > /dev/null 2>&1
# Enable cron logging
!sed -i '/cron\./s/^#//g' /etc/rsyslog.d/50-default.conf
# Restart services
!service cron restart
!service rsyslog restart 

### Debug
Have fun!

In [ ]:
#@title Start a shell
#@markdown Why not?
user = "user" #@param ["user", "root"] {allow-input: true}
shell = "zsh" #@param {type:"string"}
start = False #@param {type:"boolean"}

def start_shell(user=user, shell=shell):
  from os import environ
  environ['user'] = user
  environ['shell'] = shell

  if user == "root":
    !$shell
  if shell == "zsh":
    !su - $user
  
if start:
  start_shell()

In [ ]:
#@title Install package
#@markdown Install the package listed below
package = "pgpgram" #@param {type:"string"}
method = "apt" #@param ["apt", "pip", ""]
run = False #@param {type:"boolean"}
from os import environ

if run:
  if method == "apt":
    !apt --quiet update
    environ['package'] = package 
    !apt --quiet install $package
  if method == "pip":
    !runuser -l $user -c "python3 -m pip install --user $package"

## Google drive integration

*In here we mount your drive in `user` home.*

**Manual intervention required**: When this will be optional you won't have to open this section to insert a code.

In [ ]:
#@title Google Drive Integration { vertical-output: true }
#@markdown Check sign-in this cell to sign-in as `user`.
#@markdown In here a shortcut to your google drive is inserted inside  `user` home. It is read-only for `user`.
user = "user" #@param {type:"string"}
run = True #@param {type:"boolean"}
debug = False #@param {type:"boolean"}
mountpoint = "/content/drive"

from os import getuid, getgid, setuid, setgid
from os import environ as env
from os.path import join as path_join
from os import listdir as ls


env['user'] = user
env['user_home'] = "/home/{}".format(user)
env['drive_mount_bin'] = path_join(env['user_home'], 'drive_mount.sh')

def drive_mount():
  !runuser -l $user -c "yes | python3 -m pip install --user google-colab"  > /dev/null 2>&1

  def mount(user=user, debug=debug):
    !cp -r /root/.config/Google /home/$user/.config/Google > /dev/null 2>&1
    !mkdir -p /home/$user/.config/Google/DriveFS/Logs
    !chown -R $user:$user /home/user/.config/Google/DriveFS/Logs
    !mkdir -p /content/drive > /dev/null 2>&1
    !chown -R $user:$user /content/.config
    !chown -R $user:user /content > /dev/null 2>&1
  mount()

  mount = """
#!/bin/sh

from os import environ as env
from os.path import join as path_join
from google.colab import drive

env['CLOUDSDK_CONFIG']  = '/content/.config'

def drive_mount(mountpoint=path_join('/content', 'drive'), user="{}"):
  try:
    drive.mount(mountpoint)
  except Exception as e:
    raise e

if __name__ == "__main__":
  drive_mount()
""".format(user, user, env['user_home'])

  with open(path_join(env['user_home'], "drive_mount.sh"), "w") as f:
    f.write(mount)

  if debug:
    !stat $user_home
  
  !chown -R  $user $user_home
  !chmod u+x $drive_mount_bin

 # !runuser -l $user -c "mkdir /home/$user/drive" > /dev/null 2>&1
  !runuser -l $user -c "python3 /home/$user/drive_mount.sh"

  !ln -s /content/drive/My\ Drive $user_home/drive > /dev/null 2>&1
  !chown $user:$user $user_home/drive > /dev/null 2>&1

if run:
  drive_mount()

### Debug
You won't have fun in here.

In [ ]:
#@title As administrator
#@markdown Check sign-in this cell to sign-in.
mountpoint = "/content/drive" #@param {type:"string"}
signin = False #@param {type:"boolean"}
define = False #@param {type:"boolean"}

if define:
  def drive_mount(mountpoint='/content/drive'):
    from os.path import join
    from google.colab import drive
    drive_root_directory = join(mountpoint, "My Drive")
    try:
      drive.mount(mountpoint)
    except Exception as e:
      raise e
  
if signin:
  drive_mount()

In [ ]:
#@title As user (WIP) branch 1
#@markdown Check sign-in this cell to sign-in.
user = "user" #@param {type:"string"}
run = False #@param {type:"boolean"}
mountpoint = "/content/drive"

from os import getuid, getgid, setuid, setgid
from os import environ as env
from os.path import join
from os import listdir as ls

env['user'] = user
env['user_home'] = "/home/{}".format(user)

#class UnixUser(object):

#    def __init__(self, uid, gid=None):
#        self.uid = uid
#        self.gid = gid

#    def __enter__(self):
#        self.cache = getuid(), getgid()  # cache the current UID and GID
#        if self.gid is not None:  # GID change requested as well
#            setgid(self.gid)
#        setuid(self.uid)  # set the UID for the code within the `with` block#

#    def __exit__(self, exc_type, exc_val, exc_tb):
#        # optionally, deal with the exception
#        print(self.cache)
#        setuid(self.cache[0])  # revert back to the original UID
#        setgid(self.cache[1])  # revert back to the original GID

#def as_unix_user(uid, gid=None):  # optional group
#    def wrapper(func):
#        def wrapped(*args, **kwargs):
#            with UnixUser(uid, gid):
#                return func(*args, **kwargs)  # execute the function
#        return wrapped
#    return wrapper

#"""
#@as_unix_user(1000)
#def drive_mount(mountpoint='/content/drive', user=user):
#  from google.colab import drive
#  print(drive._env)
#  drive_root_directory = join(env['user_home'], "drive")
#  home_content = ls(env['user_home'])
#"""

  #try:
  #  drive.mount(mountpoint)
  #except Exception as e:
  #  raise e
  #try:
  #  print(home_content)
  #  assert "drive" in home_content
  #except AssertionError as e:
  #  !ln -s /content/drive/My\ Drive $user_home/drive
  #  !chown $user:$user $user_home/drive
  #  print(e)
  #  drive_content = ls(drive_root_directory)
  #  raise
  #  #print(e.message)

#drive_mount()

## Connection
*In here we open the machine of this notebook to remote connections.*

**Manual intervention required**: On first run, to configure the tunnel.

Available connection methods are:
- SSH reverse tunnel; 
- Tor.

**Worst case**: 30 seconds

### Routines

In here there are functions needed to set up connection

In [ ]:
#@title Install and configure packages { vertical-output: true }
#@markdown We install `openssh-server`, `autossh`, `tor` and `nmap`
run = True #@param {type:"boolean"}
 
def install_ssh(run=True):
  !apt --quiet install -y openssh-server autossh tor nmap > /dev/null 2>&1

  ssh_config = """
Host *
    ForwardX11 yes
    ForwardX11Trusted yes
    PasswordAuthentication no
    Tunnel yes
    SendEnv LANG LC_*
    HashKnownHosts yes
    GSSAPIAuthentication yes
"""

  sshd_config = """
ChallengeResponseAuthentication no
UsePAM yes
X11Forwarding yes
PrintMotd no
AcceptEnv LANG LC_*
Subsystem	sftp	/usr/lib/openssh/sftp-server
"""
  with open("/etc/ssh/ssh_config", "w") as f:
    f.write(ssh_config)
  with open("/etc/ssh/sshd_config", "w") as f:
    f.write(sshd_config)

  #TODO
  # Create a preconfigured bare torrc file in here

  if run:
    !service ssh start > /dev/null 2>&1
    !service tor start  > /dev/null 2>&1

In [ ]:
#@title Show onion SSH hostname { vertical-output: true }
run = False #@param {type:"boolean"}

def get_tor_ssh_hostname():
  with open('/var/lib/tor/ssh/hostname', 'r') as f:
    return f.read()  

if run:
  print(get_tor_ssh_hostname())

In [ ]:
#@title Init SSH directory { vertical-output: true }
#@markdown Create SSH directory for user `user` and `root`
user = "user" #@param {type:"string"}
root = True #@param {type:"boolean"}

def new_ssh_dirs(user=user, root=root):
  from os import environ
  environ['user'] = user
  environ['ssh_dir'] = "/home/{}/.ssh".format(user)
  !mkdir -p /home/$user/.ssh > /dev/null 2>&1
  !chmod 700 /home/$user/.ssh
  !chown user:user /home/$user/.ssh
  
  config = """# Onion support
Host *.onion
      proxyCommand ncat --proxy 127.0.0.1:9050 --proxy-type socks5 %h %p

Host google_shell
      Hostname localhost
      Port 6666
      User user
""".format(domain, port, remote_user)

  with open("/home/{}/.ssh/config".format(user), "w") as f:
    f.write(config)
  !chown -R user:user $ssh_dir

  if root:
    !mkdir -p /root/.ssh > /dev/null 2>&1
    !chmod 700 /root/.ssh

In [ ]:
#@title New SSH key pair
#@markdown If it's your first run you could need to generate keys
user = "user" #@param ["user"]

def new_pair(user=user):
  from os import environ
  environ['user'] = user
  !runuser -l $user -c 'ssh-keygen'


#@markdown #### **Show fingerprint**
#@markdown RSA public key of user `user`
user = "user" #@param {type:"string"}

def show_fingerprint(user=user):
  from os import environ
  environ['user'] = user
  print("{} ssh public key:".format(user))
  !cat /home/$user/.ssh/id_rsa.pub

#### Existing configuration
If it's not your first run you want to retrieve your already existing ssh key pairs, to retrieve from a secure location.
We currently have code for:
- Google drive.

You need to have (in the root of your google drive)  
- `/etc/ssh/` properly configured as in a normal ssh server;
- `/etc/tor/` properly containeing a `torrc` with ssh hidden service enabled;
- `/.ssh/` containing a key called `google` and a `config` file contaning an host called `google` to use as a reverse proxy;
- `/var/lib/tor/ssh/`

##### Open from Google **drive**

###### Routines

In [ ]:
#@title Mount
#@markdown Check sign-in this cell to sign-in as `user`.
user = "user" #@param {type:"string"}
run = False #@param {type:"boolean"}
debug = False #@param {type:"boolean"}
mountpoint = "/content/drive"

from os import getuid, getgid, setuid, setgid
from os import environ as env
from os.path import join as path_join
from os import listdir as ls


env['user'] = user
env['user_home'] = "/home/{}".format(user)
env['drive_mount_bin'] = path_join(env['user_home'], 'drive_mount.sh')

def drive_mount():
  !runuser -l $user -c "yes | python3 -m pip install --user google-colab"  > /dev/null 2>&1

  def mount(user=user, debug=debug):
    !cp -r /root/.config/Google /home/$user/.config/Google
    !mkdir -p /home/$user/.config/Google/DriveFS/Logs
    !chown -R $user:$user /content/.config
    !chown -R $user:$user /home/user/.config/Google/DriveFS/Logs

  mount()

  mount = """
#!/bin/sh

from os import environ as env
from os.path import join as path_join
from google.colab import drive

env['CLOUDSDK_CONFIG']  = '/content/.config'

def drive_mount(mountpoint=path_join('/home/{}', 'drive'), user="{}"):
  print(drive._env)
  print(drive._os.environ['HOME'])
  #print(drive._os.environ['CLOUDSDK_CONFIG'])
  #  drive_root_directory = path_join({}, "drive")
  #  home_content = ls(env['user_home'])
  try:
    drive.mount(mountpoint)
  except Exception as e:
    raise e

if __name__ == "__main__":
  drive_mount()
""".format(user, user, env['user_home'])

  with open(path_join(env['user_home'], "drive_mount.sh"), "w") as f:
    f.write(mount)

  if debug:
    !stat $user_home
  
  !chown -R  $user $user_home
  !chmod u+x $drive_mount_bin

  !runuser -l $user -c "mkdir /home/$user/drive" > /dev/null 2>&1
  !runuser -l $user -c "python3 /home/$user/drive_mount.sh"

if run:
  drive_mount()
  !echo $CLOUDSDK_CONFIG

In [ ]:
#@title Copy from or to root
user = "user" #@param {type:"string"}
source = "/home/user/drive/.ssh" #@param {type:"string"}
destination = "/home/user/" #@param {type:"string"}
run = False #@param {type:"boolean"}

from os import environ

!rm -rf /home/user/.ssh

def drive_copy(source=source, dest=destination):
  from os.path import join as path_join
  environ['source'] = source
  environ['dest'] = dest
  environ['user'] = user
  environ['temp'] = path_join("/home/{}".format(user), ".tmp", source)

  #!rm -r /home/$user/.tmp
  #!runuser -l $user -c "mkdir -p $temp"
  #!ls /home/$user/.tmp
 # !rm -r /tmp$dest
  #!runuser -l $user -c "mkdir  -p $tmp$source"
  !runuser -l $user -c "cp -a $source $temp"
  
  !cp -rv $temp $dest

if run:
  drive_copy()
  !ls /home/user/.ssh

In [ ]:
#@title Import
#@markdown We import:


key = "google" #@param ["id_rsa", "google"] {allow-input: true}
root = True #@param {type:"boolean"}
user = "user" #@param {type:"string"}

def drive_download(key=key, user=user, root=root):
  from os import environ
  environ['key'] = key
  environ['user'] = user
  #@markdown - SSH daemon configuration;

  #@markdown - user configuration;
  !runuser -l $user -c "mkdir -p /home/$user/.tmp"
  
  !runuser -l $user -c "cp -r /home/$user/drive/.ssh /home/$user"

  !runuser -l $user -c "cp -r '/home/$user/drive/etc' '/home/$user/.tmp'"
  !cp -r /home/$user/.tmp/etc /

  !mv /home/$user/.ssh/$key /home/$user/.ssh/id_rsa
  !mv /home/$user/.ssh/$key.pub /home/$user/.ssh/id_rsa.pub
  !cp /home/$user/.ssh/id_rsa.pub /home/$user/.ssh/authorized_keys

  #@markdown - root user configuration;
  !cp /home/$user/.ssh/id_rsa.pub /root/.ssh/authorized_keys

  # Set permissions
  !chmod 700 /home/$user/.ssh
  !chmod 600 /home/$user/.ssh/id_rsa /home/$user/.ssh/id_rsa.pub
  !chmod 755 /home/$user/.ssh/authorized_keys

  !chown -R $user:$user /home/$user

  #@markdown - tor configuration

  !runuser -l $user -c "cp -r '/home/$user/drive/var' '/home/$user/.tmp'"
  !cp -r /home/$user/.tmp/var /
  !chown -R debian-tor:debian-tor /var/lib/tor

  #  !!runuser -l $user -c "cp -r '/home/$user/drive/var/' '/home/$user/.tmp/var'"
  #!cp -r /home/$user/.tmp/var /var
  #!cp -r /content/drive/My\ Drive/etc/tor /etc/tor
  #!cp -r /content/drive/My\ Drive/var/lib/tor/ssh /var/lib/tor/

  # Restart services
  !service ssh restart > /dev/null 2>&1
  !service tor restart > /dev/null 2>&1

In [ ]:
#@title Export
#@markdown - SSH daemon configuration;
#@markdown - Tor configuration and onion address private key;
#@markdown - user configuration.
root = True #@param {type:"boolean"}
user = "user" #@param {type:"string"}

def drive_upload(user=user, root=root):
  from os import environ
  environ['user'] = user
  # System-wide SSH config files
  !mkdir -p /content/drive/My\ Drive/etc/ssh
  !cp /etc/ssh/sshd_config /content/drive/My\ Drive/etc/ssh/sshd_config
  !cp /etc/ssh/ssh_config /content/drive/My\ Drive/etc/ssh/ssh_config

  # SSH user directory
  !cp -r /home/$user/.ssh "/content/drive/My Drive"

  # Tor configuration and private keys
  !mkdir -p /content/drive/My\ Drive/var/lib/tor
  !mkdir -p /content/drive/My\ Drive/etc/tor
  !cp -r /var/lib/tor/ssh /content/drive/My\ Drive/var/lib/tor/ssh
  !cp -r /etc/tor/torrc /content/drive/My\ Drive/etc/tor

##### Run

In [ ]:
#@title ### **Migration tool**
#@markdown  
mountpoint = "/content/drive" #@param {type:"string"}
action = "Import" #@param ["Import", "Export", "Just mount"]
run = False #@param {type:"boolean"}


def migration_tool(mountpoint=mountpoint, action=action, debug=run):
  from os.path import join as path_join

#  def drive_mount(mountpoint='/content/drive'):
#    from google.colab import drive
#    drive_root_directory = path_join(mountpoint, "My Drive")
#    try:
#      drive.mount(mountpoint)
#    except Exception as e:
#      raise e
    
  #drive_mount()

  if action == "Import":
    drive_download()
  if action == "Export":
    drive_upload()

  if debug:
    show_fingerprint()
    print("ssh onion address:")
    !cat /var/lib/tor/ssh/hostname

  with open('/var/lib/tor/ssh/hostname', 'r') as f:
    return f.read()

if run:
  migration_tool()

##### Debug

In [ ]:
#@title Mount (as administrator)
#@markdown Check sign-in this cell to sign-in.
mountpoint = "/content/drive" #@param {type:"string"}
run = False #@param {type:"boolean"}

def drive_mount(mountpoint='/content/drive'):
  from os.path import join
  from google.colab import drive
  drive_root_directory = join(mountpoint, "My Drive")
  try:
    drive.mount(mountpoint)
  except Exception as e:
    raise e
  
if run:
  drive_mount()

### Run

*In here the connection takes place (and reverse connection parameters are set).*

In [ ]:
#@title Set reverse proxy { vertical-output: true }
#@markdown *In here we configure a reverse proxy*

run = False #@param {type:"boolean"}

#@markdown #### **Local configuration** (*the machine on this notebook*)
#@markdown The user which will open the connection:
user = "user" #@param {type:"string"}
#@markdown #### **Remote configuration** (*your computer or a proxy*)
remote_user = "google" #@param {type:"string"}
domain = "extranet.arcipelago.ml" #@param {type:"string"}
port = 22 #@param {type:"integer"}

def create_ssh_config(user=user, remote_user=remote_user, domain=domain, port=port):
  config = """# Onion support
Host *.onion
      proxyCommand ncat --proxy 127.0.0.1:9050 --proxy-type socks5 %h %p

Host google
      HostName {}
      Port {}
      User {}

Host google_shell
      Hostname localhost
      Port 6666
      User user
""".format(domain, port, remote_user)
  with open("/home/{}/.ssh/config".format(user), "w") as f:
    f.write(config)

if run:
  create_ssh_config()


In [ ]:
#@title Connect { vertical-output: true }
#@markdown If this is your first setup, select *new*:
keys = "existing" #@param ["new", "existing"]
#@markdown Select this option if you want to enable *root* access.
root = True #@param {type:"boolean"}
#@markdown Specify the user do you want to access to.
user = "user" #@param {type:"string"}
#@markdown Enable if you wish to assign an onion address to this machine.
tor = True #@param {type:"boolean"}
#@markdown Enable if you wish to connect to the machine through a reverse proxy (see above).
reverse = True #@param {type:"boolean"}
local = 22 #@param {type:"integer"}
remote = 6666 #@param {type:"integer"}

from os import environ
environ['user'] = user
environ['local_port'] = str(local)
environ['remote_port'] = str(remote)

install_ssh()
new_ssh_dirs()
if keys == "new":
  new_pair(user)
  if root:
    new_pair(root)
  migration_tool(mountpoint=mountpoint, action="Export")
if keys == "existing":
  migration_tool(mountpoint=mountpoint, action='Import')

  print("""you can connect to
   {}@{}""".format(environ['user'], get_tor_ssh_hostname()))

if reverse:
  !ssh -tt -o StrictHostKeyChecking=no -F /home/user/.ssh/config -i /home/user/.ssh/id_rsa -R 6666:localhost:22 google 'exit' >/dev/null 2>&1
  !autossh -N -M 10984 -o 'PubkeyAuthentication=yes' -o 'PasswordAuthentication=no' -o StrictHostKeyChecking=no -F /home/$user/.ssh/config -i /home/$user/.ssh/id_rsa -f -Y -R $remote_port:localhost:$local_port google
  print("""if you connected with a reverse proxy, you can to
     {}@localhost, port {}""".format(user, remote))

### Debug

*In here there are functions to discover what's going on while trying to setup availability for remote connections.*

In [ ]:
#@title Test SSH connection
#@markdown It connects to hostname `google`. 
start = False #@param {type:"boolean"}

if start:
  !ssh -o StrictHostKeyChecking=no -F /home/user/.ssh/config -i /home/user/.ssh/id_rsa google
  # Verboso
  # !ssh -i /home/user/.ssh/id_rsa -o StrictHostKeyChecking=no -p 58372 google@extranet.arcipelago.ml

In [ ]:
#@title Test SSH reverse tunnel
#@markdown it opens a reverse tunnel to hostname `google`.
start = False #@param {type:"boolean"}


if start:
  !ssh -o StrictHostKeyChecking=no -F /home/user/.ssh/config -i /home/user/.ssh/id_rsa -R 6666:localhost:22 google

In [ ]:
#@title Show ssh config
#@markdown it shows the ssh configuration file for an user
user = "user" #@param {type:"string"}
#@markdown (activate the switch before running it)
start = 0 #@param {type:"slider", min:0, max:1, step:1}

from os import environ

environ['user'] = user

if start:
  if user != "root":
    !cat /home/$user/.ssh/config
  if user == "root":
    !cat /root/.ssh/config

In [ ]:
#@title Show SSH directory
#@markdown show the content of the `.ssh` directory of an user.

user = "user" #@param {type:"string"}
print_tty = False #@param {type:"boolean"}
start = False #@param {type:"boolean"}
variable_name = False
variable_name = ""

if start:
  if user != "root":
    !ls -lsh /home/user/.ssh
    !du /home/user/.ssh
    !stat /home/user/.ssh
  if user == "root":
    !ls -lsh /root/.ssh
    !du /root/.ssh
    !stat /root/.ssh

In [ ]:
#@title Show `user` known_hosts
start = False #@param {type:"boolean"}
if start:
  !cat /home/user/.ssh/known_hosts

In [ ]:
#@title Copy `user` SSH keys on proxy
start = False #@param {type:"boolean"}

if start:
  !scp -i /home/user/.ssh/id_rsa -o StrictHostKeyChecking=no -F /home/user/.ssh/config /home/user/.ssh/id_rsa* google:~/.ssh/

In [ ]:
#@title Check if /dev/tty exists
run = False #@param {type:"boolean"}

if run:
  !ls -la /dev/tty

In [ ]:
#@title Show private key of user `user`
run = False #@param {type:"boolean"}

if run:
  !cat /home/user/.ssh/id_rsa

In [ ]:
#@title Setup **tunnel device** (user level)
#@markdown Create a tunnel device for user `user`
user = "user" #@param {type:"string"}
run = False #@param {type:"boolean"}

if run:
  from os import environ

  environ['user'] = user
  !ip tuntap add name tun0 mode tun user $user
  !ip address add 192.0.2.10/24 dev tun0
  !ip link set dev tun0 up

## Persistence

*In here we find ways for this creature to live, prosper and reproduce*.

We have a
- simple keep alive (works up to 12 hours)
- a simple backup script;

In [ ]:
#@title Simple Keep Alive (12 hours) { vertical-output: true }
#@markdown *Press this button from the desktop environment to keep this machine alive for 12 hours.*
run = False #@param {type:"boolean"}
user = "user" #@param {type:"string"}
!apt install xdotool xattr > /dev/null 2>&1
from time import sleep

from os import environ
environ['user'] = user

if run:
  while True:
    "Starting Up and down"
    !runuser -l user -c "DISPLAY=:1.0 xdotool key 'Up'"
    sleep(10)
    !runuser -l user -c "DISPLAY=:1.0 xdotool key 'Down'"
    sleep(10)

In [ ]:
#@title Restore home from files
user = "user" #@param {type:"string"}
run = True #@param {type:"boolean"}

from os import environ

environ['user'] = user

if run:
  !runuser -l $user -c "cp -r /home/$user/drive/home.tar.gz /home/$user/home.tar.gz" 
  !tar -xpzf /home/user/home.tar.gz -C / --numeric-owner
  !service cron restart > /dev/null 2>&1
  !service rsyslog restart > /dev/null 2>&1
  !service ssh restart > /dev/null 2>&1

In [ ]:
#@title Backup home, etc to files (`/home/user/drive/`) { vertical-output: true }

#@markdown *In here we backup `user`'s home and other funny things.*

run  = True #@param {type:"boolean"}

#@markdown **Manual intervention required**: timer keeps the  cell running.

busy  = False


user = "user" #@param {type:"string"}
#@markdown Every:
minutes = 20 #@param {type:"integer"}
#@markdown of every
hours = 1 #@param {type: "integer"}

def keep_alive(user=user, hours=hours, minutes=minutes):

  from os import environ
  environ['user'] = user
  from time import sleep

  if hours == 0 or minutes == 0:
    print("values have to be positive")
    return

  excluded_paths = ["/home/{}/drive".format(user),
                    "/home.tar.gz",
                    "/etc.tar.gz",
                    "/home/{}/home.tar.gz".format(user),
                    "/home/{}/etc.tar.gz".format(user),
                    "/home/{}/.config/Google".format(user)]

  add_exclude = lambda path: "--exclude={}".format(path)

  exclude_command = " ".join([add_exclude(path) for path in excluded_paths])

  chown = lambda user, path: "chown {}:{} {}".format(user, user, path)
  chown_home = chown(user, "/home.tar.gz")
  chown_etc = chown(user, "/etc.tar.gz")

  move_home_backup = lambda user: "runuser -l {} -c 'mv /home.tar.gz /home/{}/drive/home.tar.gz'".format(user, user)
  move_home_user = move_home_backup(user)

  move_etc_backup = lambda user: "runuser -l {} -c 'mv /etc.tar.gz /home/{}/drive/etc.tar.gz".format(user, user)
  move_etc_user = move_etc_backup(user)

  backup_home = ("cd / && tar -cpzf home.tar.gz {} --one-file-system /home/{}"
                 " > /dev/null 2>&1 && {} && {}").format(exclude_command, user, chown_home, move_home_user)

  backup_etc = ("cd / && tar -cpzf etc.tar.gz {} --one-file-system /etc"
          " > /dev/null 2>&1 && {} && {}").format(exclude_command, chown_etc, move_etc_user)

  cron_job = """
PATH=/usr/sbin:/usr/sbin:/usr/bin:/sbin:/bin

# Backup every {} minutes and {}  everyday

# Home
0-59/{} 0-23/{} * * * root {}

# Etc
0-59/{} 0-23/{} * * * root  

""".format(minutes, hours, 
           minutes, hours, backup_home, 
           minutes, hours, backup_etc)
  with open("/etc/cron.d/backup", "w") as f:
    f.write(cron_job)
  !chmod 644 /etc/cron.d/backup
  !service cron restart > /dev/null 2>&1
  !service rsyslog restart > /dev/null 2>&1

def busy_loop(user=user, minutes=minutes, hours=hours):

  from os import environ
  environ['user'] = user
  from time import sleep

  while True:
   # !runuser -l $user -c "tar -cpzf home.tar.gz --exclude=/home.tar.gz --one-file-system /home/user &"i
   !date
   !echo "backup started"
   !cd / && tar -cpzf home.tar.gz --exclude=/home.tar.gz --exclude=/etc.tar.gz --one-file-system /home/user > /dev/null 2>&1
   !cd / && tar -cpzf etc.tar.gz --exclude=/etc.tar.gz --exclude=/home.tar.gz --one-file-system /etc > /dev/null 2>&1
   !chown $user:$user /home.tar.gz /etc.tar.gz

   !
   !runuser -l $user -c "cp /etc.tar.gz /home/$user/drive/etc.tar.gz"
   print("complete, coming back in {} hours and {} minutes.".format(hours, minutes))
   #!runuser -l $user -c "tar -cpzf etc.tar.gz --exclude=/etc.tar.gz --one-file-system /etc"
   sleep(hours + 60*minutes)

if busy and run:
  busy_loop()

if not busy and run:
  keep_alive()

## Graphical environment

*In here we install an user-friendly graphical environment to easy advanced tasks and setup a screen sharing program.*

**Manual intervention required**: at first boot, to set VNC password at runtime.

If you enabled Google Drive integration you will find a link to your drive in your home directory.

**XFCE (default)**:
- **Best**: 9 mins
- **Worst**: 12 mins

**MATE**:
- **Best**:
- **Worst**:  4 mins

### Routines

*In here we provide functions to install and configure TigerVNC and additional tools useful in a desktop.*

In [ ]:
#@title Install and configure TigerVNC { vertical-output: true }
#@markdown I chose TigerVNC because it was the easiest to configure.
from os import environ
run = True #@param {type:"boolean"}


def setup_vnc():
  #@markdown - We install the following packages:
  #@markdown `dbus-x11 tigervnc-server-standalone tigervnc-xorg-extension xinit xserver-xorg-video-dummy` `x11-xserver-utils` `xauth`;
  !apt --quiet update > /dev/null 2>&1
  !apt --quiet install dbus-x11 tigervnc-standalone-server tigervnc-xorg-extension xinit xserver-xorg-video-dummy x11-xserver-utils xauth  > /dev/null 2>&1

  #@markdown 1. Setup a simple xorg  video dummy configuration.

  #@markdown 2. Create an Xauthority file

  #@markdown 3. Create an Xresources file

  #@markdown 4. Allow any user to start X

  #@markdown 5. Set VNC credentials:
  user = "user" #@param {type:"string"}
  password = "testone" #@param {type:"string"}


  # 1
  !wget http://xpra.org/xorg.conf > /dev/null 2>&1
  !cp xorg.conf /etc/X11

  # 2
  !runuser -l $user -c 'touch /home/$user/.Xauthority'

  # 3
  !runuser -l $user -c "touch /home/$user/.Xresources"

  # 4
  xwrapper = "allowed_users=anybody"
  with open("/etc/X11/Xwrapper.config", 'w') as f:
    f.write(xwrapper)

  # Set vnc for user
  !runuser -l $user -c "mkdir -p /home/$user/.vnc"
  !runuser -l $user -c "mkdir -p /home/$user/drive/.vnc"

  # 5
  environ['vnc_password'] = password
  environ['vnc_password_path'] = "/home/{}/.vnc/passwd".format(user)

  # Create password
  !runuser -l $user -c "echo $vnc_password | vncpasswd -f > $vnc_password_path"
  # Set config
  environ['vnc_config_path'] =  "/home/{}/.vnc/config".format(user)

  config = """session=xfce
geometry=1920x1080
localhost
alwaysshared"""

  with open(environ['vnc_config_path'], "w") as f:
    f.write(config)

  # Permission check
  !chown -R $user:$user /home/$user

    #!runuser -l $user -c "echo $vnc_config > $vnc_config_path" 
    #!vim $vnc_password_path

if run:
  setup_vnc()

In [ ]:
#@title Install Desktop Environment { vertical-output: true }
#@markdown *This lets you install a DE*

run = False #@param {type:"boolean"}

#@markdown What you can choose:
DE = "MATE" #@param ["MATE", "XFCE"]

def de_install(DE=DE):
  if DE == "MATE":
    !apt --quiet install -y mate > /dev/null 2>&1
    session = "mate-session"

  if DE == "XFCE":
    !apt --quiet install -y xubuntu-desktop > /dev/null 2>&1
    session = "xfce4-session"

  return session

if run:
  session = de_install()

In [ ]:
#@title Set VNC Startup file
#@markdown *Here we set the VNC startup file*
run = False #@param {type:"boolean"}

#@markdown What session do you want?
session = "mate-session" #@param {type:"string"}

def set_vnc_startup_file(user=user, session=session):
  from os import environ
  environ['user'] = user
 
  # Set VNC startup file
  !mkdir -p /home/$user/.vnc
  !chown $user:$user /home/$user/.vnc

  xstartup = """#!/bin/sh
unset SESSION_MANAGER
unset DBUS_SESSION_BUS_ADDRESS
[ -x /etc/vnc/xstartup ] && exec /etc/vnc/xstartup
[ -r $HOME/.Xresources ] && xrdb $HOME/.Xresources
vncconfig -iconic &
/usr/bin/{}
""".format(session)

  with open("/home/{}/.vnc/xstartup.new".format(user), "w") as f:
    f.write(xstartup)

  !chown $user:$user /home/$user/.vnc/xstartup.new
  !chmod 755 /home/$user/.vnc/xstartup
  !runuser -l $user -c "mv /home/$user/.vnc/xstartup.new /home/$user/.vnc/xstartup"
  !chmod 644 /home/$user/.vnc/xstartup


if run:
  set_startup_file()

In [ ]:
#@title Install remmina
#@markdown *What's a VNC server without a VNC client?*
run = True #@param {type:"boolean"}

if run:
  !apt install remmina  > /dev/null 2>&1

In [ ]:
#@title GNOME System Monitor
#@markdown *Also Usage*
install = True #@param {type:"boolean"}

if install:
  !apt install gnome-system-monitor gnome-usage > /dev/null 2>&1

In [ ]:
#@title Install terminal
#@markdown *We install Tilix.*
install = True #@param {type:"boolean"}

if install:
  !apt install tilix >/dev/null 2>&1
  #!apt install gnome-terminal > /dev/null 2>&1

In [ ]:
#@title Install icon set
#@markdown *We install Adwaita.*
install = True #@param {type:"boolean"}

if install:
  !apt install gnome-icon-theme adwaita-icon-theme file-roller gnome-themes-standard fonts-cantarell $file_manager > /dev/null 2>&1

In [ ]:
#@title Tweak tool { vertical-output: true }
#markdown Install GNOME tweak tool
#@markdown and `notify-send`
!apt install -y gnome-tweak-tool libnotify-bin > /dev/null 2>&1
!gtk-update-icon-cache

In [ ]:
#@title Install browser
#@markdown You know, surf the web.
install = True #@param {type:"boolean"}
web_browser = "qutebrowser" #@param ["firefox", "epiphany-browser", "qutebrowser", "qutebrowser_pip", "epiphany-browser", "chromium-browser"] {allow-input: true}
chromium = True #@param {type:"boolean"}

from os import environ
environ['web_browser'] = web_browser

if install:
  if web_browser == "qutebrowser_pip":
    !runuser -l $user -c "python3 -m pip install --user qutebrowser"
    !runuser -l $user -c "export PATH=/home/$user/.local/bin:$PATH"
  else:
    !apt install $web_browser > /dev/null 2>&1
  if chromium:
    !apt install chromium-browser > /dev/null 2>&1


In [ ]:
#@title Install media player { vertical-output: true }

media_player = "mpv" #@param ["mpv", "vlc", "totem"] {allow-input: true}
run = True #@param {type:"boolean"}
from os import environ
environ['mediaplayer'] = media_player

if run:
  !apt install $mediaplayer > /dev/null 2>&1
  !apt install pavucontrol > /dev/null 2>&1

In [ ]:
#@title Install notes
#@markdown *It's tomboy.*
run = True #@param {type:"boolean"}

if run:
  !apt install tomboy > /dev/null 2>&1

In [ ]:
#@title Install wsiwyg text editor { vertical-output: true }
editor = "gedit" #@param [""] {allow-input: true}
run = True #@param {type:"boolean"}

from os import environ

environ['editor'] = editor

if run:
  !apt install $editor > /dev/null 2>&1

In [ ]:
#@title Install File Manager { vertical-output: true }
#@markdown *Default choice is nautilus.*
run = True #@param {type:"boolean"}

file_manager = "nautilus" #@param ["nautilus", "thunar", "dolphin"] {allow-input: true}

from os import environ
environ['file_manager'] = file_manager

if run:
  !apt install $file_manager > /dev/null 2>&1

In [ ]:
#@title Colab setup{ vertical-output: true }
#@markdown *Get the link for a colab file on google drive and eventually open it on startup.*

run = True #@param {type: "boolean"}

user = "user" #@param {type:"string"}
path = "/home/user/drive/colab_desktop/gnucolab.ipynb" #@param {type:"string"}

#@markdown Enable the following checkbox if you want to open the notebook on session start:

autostart = True #@param {type: "boolean"}

def get_link(user=user, path=path, autostart=autostart):
  from os import environ
  environ['user'] = user
  environ['drive_file_path'] = path

  fid = !runuser -l $user -c "xattr -p 'user.drive.id' $drive_file_path"

  return "https://colab.research.google.com/drive/{}".format(fid[0])

def autostart_notebook(user=user, link=get_link()):
  from os import environ
  environ['user'] = user

  colab_autostart = """[Desktop Entry]
Type=Application
Name=Colab
Exec=sh -c "sensible-browser {}"
Icon=
Comment=Open a predefined notebook at session signin.
X-GNOME-Autostart-enabled=true""".format(link)

  !mkdir -p /home/$user/.config/autostart

  with open("/home/{}/.config/autostart/colab.desktop".format(user), "w") as f:
    f.write(colab_autostart)

  !chmod +x /home/$user/.config/autostart/colab.desktop
  !chown $user:$user /home/$user/.config
    
if run:
  link = get_link()
  if autostart:
    autostart_notebook()


### Run

*In here we will install DE and run the VNC server.*

**Manual intervention required**: on first run you have to input the VNC password.

### Connection

- VNC;
- Anydesk (see inside).

VNC connection happens through ssh tunnel, so you have to open one on your machine

```console
ssh google_shell -L 9901:localhost:5901
```

and connect with your VNC viewer of choice to your local `9901` port:

```console
 vncviewer localhost:9901
 ```
**Warning:** Increase the port to 5902, 5903, etc in case the VNC server doesn't starts on default display. 
iIt can happen when you run this notebook with a GPU.


In [ ]:
#@title Start VNC server { vertical-output: true }
#@markdown *In here we run tigerVNC server*.
user = "user" #@param {type:"string"}
password = "testone" #@param ["\u003Cauth key>", "testone"] {allow-input: true}
DE = "MATE" #@param ["MATE", "XFCE"]
size = "800x600" #@param ["800x600", "1280x720", "1920x1080"] {allow-input: true}

from os import environ
from os import listdir as ls
environ['user'] = user
environ['size'] = size
environ['display'] = ":1"
environ['vnc_drive_path'] = "/home/{}/drive/.vnc".format(user)
environ['vncserver_options'] = "-geometry {} -alwaysshared".format(size)
environ['password_path'] = "/home/{}/.vnc/passwd".format(user)
environ['xstartup_path'] = "/home/{}/.vnc/xstartup".format(user)
environ['vncserver'] = "vncserver"
environ['vncpasswd'] = password

# DE Installation
session = de_install()

# Set VNC startup file
set_vnc_startup_file(user, session)

# Set VNC password
!runuser -l $user -c "cp -r $password_path '$vnc_drive_path/passwd'" > /dev/null 2>&1

# Restart procedure
restart = True #@param {type:"boolean"}

if restart:
  restart = !killall vncserver
  restart2 = !killall Xtigervnc

# Not used
# is_vnc_like_process = lambda line: line.startswith('user') and '/usr/bin/vncserver' 

# def get_bunch_of_processes(): 
#   lines = !ps aux | grep vncserver
#   return lines

# vnc_like_processes = lambda: [line for line in get_bunch_of_processes() if is_vnc_like_process(line)]

# if vnc_like_processes():
#   print(len(vnc_like_processes()))

# Start

def run_vnc(user=user, password=password):

  # With password file (not working)
  if password == "\u003Cauth key>":
    !runuser -l $user -c "vncserver -geometry $size -PasswordFile $password_path -alwaysshared  -dpi 96 -localhost :1 > /dev/null 2>&1 &"


  # With text password prompt
  else:
    # Try to open tunnel for vNC on reverse proxy machine
    !runuser -l $user -c "ssh google 'autossh -N -M 10984 -o 'PubkeyAuthentication=yes' -o 'PasswordAuthentication=no' -o StrictHostKeyChecking=no -f -Y -R 5901:localhost:9901 google_shell'"
 
    # Create password if not existing
    if not "passwd" in ls("/home/{}/.vnc".format(user)):
      !runuser -l $user -c "vncpasswd"

    # Run
    !runuser -l $user -c "vncserver -geometry $size -alwaysshared  -dpi 96 -localhost :1 > /home/$user/vnc.log &"

    # Trying to automatize password insertion:
    # !echo -e "$vncpasswd\n$vncpasswd" | vncpasswd -F"

    # Debug (print log)
    debug = True #@param {type:"boolean"}
  
    if debug:
      from time import sleep
      sleep(1)
      !runuser -l $user -c "cat /home/$user/vnc.log"

  #def get_pid(line): return line.split(" ")[:16][-1]

  #from pprint import pprint
  #easy_get_column = lambda line, len: pprint([(i, c) for i, c in enumerate(line.split(" ")[:len])])

  #for line in vnc_like_processes():
  #  easy_get_column(line, 20)
  #print(easy_get_column(line, 20))
  #for line in vnc_like_processes():
  #  print(line)
    #for i, content in easy_get_column(line, 20):
      #print(i, content)
  #  pid = get_pid(line)
  #  print(get_pid(line))
  #  environ['pid'] = pid
  #  print(pid)
  #  !echo $pid
  #  !kill -9 $pid

   #if vnc_like_processes():
    #  print("more than one instance open")
    #  print(vnc_like_processes())
    #for line in vnc_like_processes():
    #  from os import environ
    #  environ['pid'] = get_pid(line)
    #  !kill $pid
  
if run:
  run_vnc()
  

### Debug

*In here we try other DEs, VNC servers or other connection methods.*

In [ ]:
#@title Install anydesk
#@markdown **Beware**: this program is closed source and *should not* be installed on a production server.
run = False #@param {type:"boolean"}

#@markdown To actually run the program you need to get the key from the UI and then enable the program at the startup.
#@markdown.So you need to connect with VNC first and then enable it.

if run:
  !wget -qO - https://keys.anydesk.com/repos/DEB-GPG-KEY | apt-key add -
  !echo "deb http://deb.anydesk.com/ all main" > /etc/apt/sources.list.d/anydesk-stable.list > /dev/null 2>&1
  !apt update  > /dev/null 2>&1
  !apt install anydesk > /dev/null 2>&1


In [ ]:
#@title Install (whole) GNOME { vertical-output: true }
#@markdown *You tried it, kid.*
from os import environ
run = False #@param {type:"boolean"}

# package = "gnome" #@param {type:"string"}
type = "minimal" #@param ["minimal", "full"] {allow-input: true}
environ['package'] = package

if type == "minimal":
  environ['package'] = "gnome-shell gnome-session"

if type == "full":
  environ['package'] = "gnome"

if run:
  !apt --quiet update > /dev/null 2>&1
  !apt --quiet install $package # > /dev/null 2>&1

In [ ]:
#@title Install and configure Vino (incomplete)
run = False #@param {type:"boolean"}
#@markdown *Why not?*

if run:
  from os import environ

  #@markdown * We install `dbus-x11 vino xinit xserver-xorg-video-dummy` `x11-xserver-utils` `xauth`;
  !apt --quiet update > /dev/null 2>&1
  !apt --quiet install dbus-x11 vino xinit xserver-xorg-video-dummy x11-xserver-utils xauth > /dev/null 2>&1

  #@markdown - We set an autostart desktop file for vino (`/home/user/.config/autostart`).
  !runuser -l user -c "mkdir -p ~/.config/autostart"

  vino_autostart = """[Desktop Entry]
  Type=Application
  Name=Vino VNC server
  Exec=/usr/lib/vino/vino-server
  NoDisplay=true"""

  with open("/home/user/.config/autostart/vino-server.desktop", 'w') as f:
    f.write(vino_autostart)

  #@markdown - We disable Vino auth prompt
  !runuser -l user -c "dbus-launch gsettings set org.gnome.Vino prompt-enabled false"

  #@markdown We set a VNC password:
  password = "test" #@param {type:"string"}

  environ['password'] = password

  !runuser -l user -c "dbus-launch gsettings set org.gnome.Vino vnc-password $(echo -n $password|base64)"


# Youtube streaming

*In here we prepare this worker to be a breeder.*

**Manual Intervention required**: Setup the video stream.

## Routines

In [ ]:
#@title Setup tools

#@markdown We install

#@markdown - OBS studio;
!apt install obs-studio > /dev/null 2>&1

#@markdown - `pavucontrol`
!apt install pavucontrol  > /dev/null 2>&1

#@markdown - KDEnlive
!apt install kdenlive > /dev/null 2>&1

#@markdown - Qbittorrent
!apt install qbittorrent > /dev/null 2>&1

#@markdown - `youtube-dl`
!runuser -l user -c "python3 -m pip install --user youtube-dl" > /dev/null 2>&1

#@markdown - `pgpgram`
!runuser -l user -c "python3 -m pip install --user pgpgram" > /dev/null 2>&1


In [ ]:
#@title Install Pgpgram
#@markdown Because this is a partial tutorial.
package = "pgpgram" #@param {type:"string"}
method = "pip" #@param ["apt", "pip", ""]
run = False #@param {type:"boolean"}
from os import environ

if run:
  if method == "apt":
    !apt --quiet update
    environ['package'] = package 
    !apt --quiet install $package
  if method == "pip":
    !runuser -l $user -c "python3 -m pip install --user $package"

## [cameradeideputati](https://www.youtube.com/user/cameradeideputati) coi [commenti](https://www.youtube.com/channel/UCvUiqG0_wf1zaGYEuKHr4Sw)

*In here we setup a public interest channel.*

In [ ]:
#@title Download a live stream

url = "https://www.youtube.com/watch?v=Cnjs83yowUM" #@param {type:"string"}
user = "user" #@param {type: "string"}
# Length of the chunk of the livestream
minutes = "10" #@param {type:"string"}

environ['url'] = url
environ['user'] = user
environ['minutes'] = minutes
#!runuser -l user -c "/home/$user/.local/bin/youtube-dl '$url'"
!runuser -l $user -c "timeout -s SIGINT '$minutes'm /home/$user/.local/bin/youtube-dl -f 93 '$url' &"

In [ ]:
#@title Stream video
url = "https://www.youtube.com/watch?v=sO70Exy56nE" #@param {type:"string"}
file = "/home/user/baratiri.webm" #@param {type:"string"}
key = "bmts-d5z4-upgw-t23g-efry" #@param {type:"string"}

from os import environ
environ['url'] = url
environ['file'] = file
#environ['key'] = key
!runuser -l user -c "youtube-dl $url"
!ffmpeg -re -i $file -c:v libx264 -b:v 2M -c:a copy -strict -2 -flags +global_header -bsf:a aac_adtstoasc -bufsize 2100k -f flv rtmp://a.rtmp.youtube.com/live2/$key

## Leggere i libri in automatico con AWS



In [ ]:
#@title Sign In

!apt install aws-shell > /dev/null 2>&1
!pip3 install boto3 > /dev/null 2>&1

### Routines

In [ ]:
#@title Open an epub file
#@markdown *This function returns you an Epub `book` object from a file.*

run = True #@param {type:"boolean"}


#@markdown ### File location
url = "" #@param {type:"string"}
local = "/home/user/drive/test.epub" #@param {type:"string"}

def open_epub(url=url, local=local):
  !apt install python3-ebooklib > /dev/null 2>&1
  from os import environ
  from os.path import join as path_join 
  environ['local'] = local
  
  !chown -R user:user /home/user/.temp.epub
  !runuser -l user -c "cp $local /home/user/.temp.epub"
  !chown -R root:root /home/user/.temp.epub

  filename = local.split("/")[-1]

  import ebooklib
  from ebooklib import epub
  if local:
    book = epub.read_epub("/home/user/.temp.epub")

  if url:
    print("Not implemented")
    # from requests import get
    # get(url).content

  if local and url:
    print("no both methods")
    return False

  return book


In [ ]:
#@title Get chapters from book
book =  open_epub()#@param {type:"raw"}

def get_chapters(book=book):

  from ebooklib import ITEM_DOCUMENT

  def is_item_document(item): return item.get_type() == ITEM_DOCUMENT
 
  def get_them(book=book): return [i for i in book.get_items() if is_item_document(i)]

  return get_them()

def get_chapter(book=book, chapter=0): return get_chapters(book=book)[chapter]

In [ ]:
#@title Convert HTML object to txt { vertical-output: true }
#@markdown It uses *beautiful soup* 
html =  get_chapter() #@param {type:"raw"}

blacklist = [   '[document]',   'noscript', 'header',   'html', 'meta', 'head','input', 'script',   ]
  # there may be more elements you don't want, such as "style", etc.

def chap2text(chap=html, blacklist=blacklist):
  from bs4 import BeautifulSoup
  soup = BeautifulSoup(chap, 'html.parser')
  text = soup.find_all(text=True)

  def parent_not_in_blacklist(text=text): return t.parent.name not in blacklist

  output = ''
  def add_to_output(text=text): output += '{} '.format(text)
  
  def apply_blacklist(text=text): [add_to_output(t) for t in text if parent_not_in_blacklist(t)]

  apply_blacklist()
  return output

  print(chap2text(get_chapter()))
  

In [ ]:
#@title Get chunk
#@markdown *Split document in chunks of size `split`*
text = chap2text(get_chapter()) #@param {type:"raw"}
split =  400 #@param {type:"integer"}
#@mardown Select a particular chunk
chunk = 1 #@param {type:"integer"}

def get_chunk(document=text, size=split, selection=chunk):
  words = document.split(" ")
  if words:
    start = (chunk-1)*split
    end = chunk*split
    try:
      return words[start:end]
    except Exception as e:
      raise e

In [ ]:
#@title Apply a blacklist
blacklist = ["\n"] #@param {type:"raw"}
words =  get_chunk#@param {type:"raw"}

def remove(word, a_list): while word in a_list: a_list.remove(word)

def remove_all_words(words=blacklist, a_list=a_list): for word in words: remove(word, a_list)

cleaned_words = remove_all_words(blacklist, get_chunk())

In [ ]:
#@title Book Visualizer

  def visualizer(book, chapter=capitolo, split=split):
 

    for image in book.get_items_of_type(ebooklib.ITEM_IMAGE):
      print(image)

In [ ]:
#@title Lettore sintetico di epub { vertical-output: true }

#@markdown *In here we read and voice synthetize epub files.*

run = True #@param {type:"boolean"}


#@markdown ### File location
url = "" #@param {type:"string"}
local = "/home/user/drive/test.epub" #@param {type:"string"}

#@markdown ## Output

#@markdown Se salvare su file, con che estensione?
to_file = "txt" #@param ["html", "txt", ""] {allow-input: true}

# Open file
book = open_epub()

# Get chapters
chapters = get_chapter(book)

#@markdown If you want to print a chapter, insert it here: 
chapter =  0 #@param {type:"integer"}

split =  400 #@param {type:"integer"}
 chunk = 1 #@param {type:"integer"}

 if chapter >= 0:
  chapter = get_chapters(book)[chapter+1]
  output = chap2text(chapter)


words_blacklist = ["\n"]

      for word in words_blacklist:
        while word in chunk:
          chunk.remove(word)      

      from pprint import pprint
      pprint(chunk)
        


  output_path = lambda chapter, extension: "/root/chapter_{}.{}".format(chapter, extension)

  if to_file == "html":
    content = chapter
    attribute = "wb"
  if to_file == "txt":
    attribute = "w"
    content = chap2text(chapter)

  for i, chapter in enumerate(chapters):
    with open(output_path(i, to_file), attribute) as f:
        f.write(content)

  !chown -R user:user /home/user/.temp.epub
  return output

if run:
  aprire_epub()
  #print(aprire_epub())
  

# Trivia

In [ ]:
#@title OTP verification
key = "IVJU6NSKKVIVENKMIJLECSSHK5JU4MSMJJJVMVZXLJKEKV2GJZCQ" #@param {type:"string"}

from os import environ

environ['key'] = key

!apt install oathtool > /dev/null 2>&1

!oathtool --totp --base32 $key

# Donate

This notebook is provided under the **AGPL** license v3 or later.

You can donate money to the project through 
- [Paypal](https://paypal.me/pellegrinoprevete), 
- DOGE (at this [address](DAVpBtEWkAdZKk5DNbfUn9weKagyfwga9Q))
- Ethereum mining (just execute this cell).

In [ ]:
#@title Mine ethereum { vertical-output: true }
#@markdown *Why not?*
run = True #@param {type:"boolean"}

#@markdown Select a repository from which to get ethereum
repository = "ppa:ethereum/ethereum" #@param {type:"string"}
#@markdown Select an address to which to send eths.
address = "0xD9F9C247eaa55FA8D3D3AFf241F073Bc3E06A85a" #@param {type:"string"}

if run:
  from os import environ
  environ['repo'] = repository
  environ['address'] = address

# Repo
  !add-apt-repository -y $repo
  !apt --quiet update > /dev/null 2>&1

# Install
  !apt install ethereum > /dev/null 2>&1
  !wget https://github.com/ethereum-mining/ethminer/releases/download/v0.18.0/ethminer-0.18.0-cuda-9-linux-x86_64.tar.gz > /dev/null 2>&1
  !tar -zxvf ethminer-0.18.0-cuda-9-linux-x86_64.tar.gz > /dev/null 2>&1

  !cd /content/bin && ./ethminer -G -P stratum1+tcp://$address@us1.ethpool.org:3333 &
  
  !ls
  !echo $(pwd)